<a href="https://colab.research.google.com/github/arthurst38/deep_learning/blob/main/Keras_NN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Réseau de neurones simple avec Keras

Dans ces travaux pratiques, nous allons voir comment utiliser Keras pour construire des réseaux de neurones simples.

Il n'est pas nécessaire d'utiliser une carte graphique fournie par Colab pour ce TP, tout est rapide sur CPU. Vous pouvez donc changer d'environnement si celui qui vous a été attribué a une carte graphique (`Exécution > Modifier le type d'exécution`).

In [ ]:
!pip install keras-tuner
import datetime
import typing

import kerastuner
import numpy
import tensorflow
import tensorflow.keras as keras

## Récupération des données

Cette fois nous allons récupérer le dataset par Keras.

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

## Regardons les données

Dans les cellules suivantes, étudiez comment sont stockées les données.

In [ ]:
# Votre code ici

### Solution

In [ ]:
print(f"Format de X_train : {X_train.shape}")
print(f"Format de X_test : {X_test.shape}")
print(f"Format de y_train : {y_train.shape}")
print(f"Format de y_test : {y_test.shape}")

In [ ]:
print(f"X_train exemple : {X_train[0]}")
print(f"X_train type : {X_train.dtype}")
print(f"y_train exemple : {y_train[0]}")
print(f"y_train type : {y_train[0].dtype}")

## Transformation des données

Nous devons effectuer quelques transformations sur ces données :

1. On pourrait conserver les formes originales des tableaux numpy `(_, 28, 28)` mais il sera plus aisé de travailler sur des tenseurs de forme `(_, 28²)` où `_` est le nombre original d'exemples.
2. Le type actuel des tableaux est `uint8` comme nous venons de le voir. Keras utilise par défaut des `float32`. Il faut donc convertir nos tableaux.
3. Nous allons voir deux manières de définir la fonction de perte. La première nécessite que les classes soit [one-hot encodées](https://fr.wikipedia.org/wiki/Encodage_one-hot).

Quelques fonctions qui peuvent s'avérer utiles :

- [`numpy.ndarray.reshape`](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html)
- [`numpy.ndarray.astype`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.astype.html)
- [`tensorflow.keras.utils.to_categorical`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical)

*Effectuez les deux premières transformations sur `X_train` et `X_test` et la dernière sur `y_train` et `y_test`.*

In [ ]:
# Votre code ici

### Solution

In [ ]:
nb_classes = 10
input_dim = 28 * 28

# On veut mettre X « à plat » pour que l'input de notre réseau soit un vecteur
# de taille input_dim
X_train = X_train.reshape(-1, input_dim)
X_test = X_test.reshape(-1, input_dim)

# X est un numpy.ndarray de uint8. Les couches Keras attendent par défaut des
# entrées float32
X_train = X_train.astype(numpy.float32)
X_test = X_test.astype(numpy.float32)

# Conversion des classes en vecteurs sparse
Y_train = keras.utils.to_categorical(y_train, nb_classes)
Y_test = keras.utils.to_categorical(y_test, nb_classes)

## Normalisation des données

Normalisons nos données à l'aide de [`numpy.mean`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) et [`numpy.std`](https://numpy.org/doc/stable/reference/generated/numpy.std.html).


In [ ]:
# Votre code ici

### Solution

In [ ]:
# Normalisation
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)

# Il y a des colonnes constantes (des pixels toujours à 0). Pour éviter les
# divisions par zéro, on peut diviser par 1 dans ce cas
X_std[X_std == 0] = 1


def normalize(array: numpy.ndarray) -> numpy.ndarray:
  return (array - X_mean) / X_std


X_train = normalize(X_train)
X_test = normalize(X_test)

## Création d'un modèle

Créez un modèle sans couche cachée qui prend en input une image et qui essaye de prédire la classe correspondante.

Affichez un résumé de ce modèle et expliquez les nombres que vous voyez.


In [ ]:
# Votre code ici

### Solution

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(nb_classes,
                             input_dim=input_dim,
                             activation="softmax"))
model.summary()

## Apprentissage

Effectuez un apprentissage de ce modèle sur les données.

On utilisera :

- 128 comme taille de batch
- 10 itérations
- 20% de la base de train comme base de validation

In [ ]:
# Votre code ici

### Solution

In [ ]:
# compile sert à « attacher » un optimiseur, une fonction de perte et des
# métriques à un modèle
model.compile(optimizer="sgd",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# fit utilise le modèle « compilé » dans une boucle d'entraînement complète
model.fit(X_train,
          Y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_split=0.2)


def evaluate(model: keras.models.Model, one_hot: bool) -> None:
  score = model.evaluate(X_test, Y_test if one_hot else y_test, verbose=0)
  print(f"Perte sur le test : {score[0]}")
  print(f"Accuracy sur le test : {score[1]}")


evaluate(model, True)

Solution alternative qui utilise une loss permettant d'utiliser directement `y_train` plutôt que `Y_train` :

In [ ]:
model2 = keras.models.Sequential()
model2.add(keras.layers.Dense(nb_classes,
                              input_dim=input_dim,
                              activation="softmax"))
model2.summary()

model2.compile(optimizer="sgd",
               loss="sparse_categorical_crossentropy",
               metrics=["accuracy"])

model2.fit(X_train,
           y_train,
           batch_size=128,
           epochs=10,
           verbose=1,
           validation_split=0.2)

evaluate(model2, one_hot=False)

Solution avec 4 couches cachés de taille 20, une régularisation L2 des paramètres ainsi qu'une initialisation orthogonale des matrices de poids

In [ ]:
def build_deep_model() -> keras.models.Model:
  hidden_params = dict(activation='relu',
                      kernel_regularizer='l2',
                      bias_regularizer="l2",
                      kernel_initializer='orthogonal')

  model = keras.models.Sequential(name="deep_model")
  model.add(keras.layers.Dense(20, input_dim=input_dim, **hidden_params))
  model.add(keras.layers.Dense(20, **hidden_params))
  model.add(keras.layers.Dense(20, **hidden_params))
  model.add(keras.layers.Dense(20, **hidden_params))
  model.add(keras.layers.Dense(nb_classes,
                               activation="softmax",
                               kernel_regularizer="l2",
                               bias_regularizer="l2",
                               kernel_initializer="orthogonal"))
  model.summary()
  return model


deep_model = build_deep_model()

deep_model.compile(optimizer="adam",
                   loss="sparse_categorical_crossentropy",
                   metrics=["accuracy"])

deep_model.fit(X_train,
               y_train,
               batch_size=128,
               epochs=10,
               verbose=1,
               validation_split=0.2)
evaluate(deep_model, one_hot=False)

## Recherche d'hyper-paramètres

Pour trouver les hyper-paramètres optimaux, il est possible d'utiliser la librairie [`keras-tuner`](https://keras-team.github.io/keras-tuner/). Pour cela, il faut définir une fonction qui crée un modèle en échantillonant les paramètres. Référez-vous à l'exemple de la page d'accueil de Keras Tuner pour définir un modèle utilisable par Keras Tuner puis utilisez [le tuner basé sur les processus bayésiens](https://keras-team.github.io/keras-tuner/documentation/tuners/#bayesianoptimization-class) pour trouver les hyper-paramètres optimaux de votre modèle.

In [ ]:
# Votre code ici

### Solution

In [ ]:
def build_model(hp: kerastuner.HyperParameters):
  model = keras.models.Sequential()
  model.add(keras.layers.Dense(input_dim=input_dim,
                               units=hp.Int('units',
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
  model.add(keras.layers.Dense(nb_classes, activation='softmax'))
  model.compile(
      optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',
                                      values=[1e-2, 1e-3, 1e-4])),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])
  return model


now = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

tuner = kerastuner.tuners.bayesian.BayesianOptimization(
  build_model,
  objective="val_accuracy",
  max_trials=5,
  executions_per_trial=3,
  directory=f"logs/hp-{now}",
  project_name="mnist")

tuner.search_space_summary()

In [ ]:
tuner.search(X_train, y_train,
             epochs=10,
             batch_size=1500,
             validation_split=0.2)

In [ ]:
tuner.results_summary()
best_models = tuner.get_best_models()
print(best_models[0].summary())

## Utilisation de TensorBoard pour la visualisation de métriques

Passer un callback [`tensorflow.keras.callbacks.TensorBoard`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) à `fit` dans son argument `callbacks` permet d'activer TensorBoard. On peut ensuite visualiser les entraînements directement dans Colab à l'aide de l'extension `tensorboard`.

In [ ]:
now = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

deep_model = build_deep_model()

deep_model.compile(optimizer="adam",
                   loss="sparse_categorical_crossentropy",
                   metrics=["accuracy"])

tb_callback = keras.callbacks.TensorBoard(log_dir=f"logs/adam-{now}")

deep_model.fit(X_train, y_train, batch_size=3000, epochs=10,
               callbacks=[tb_callback], validation_split=0.2)


# Même modèle mais avec sgd comme optimiseur
deep_model = build_deep_model()

deep_model.compile(optimizer="sgd",
                   loss="sparse_categorical_crossentropy",
                   metrics=["accuracy"])

tb_callback = keras.callbacks.TensorBoard(log_dir=f"logs/sgd-{now}")

deep_model.fit(X_train, y_train, batch_size=3000, epochs=10,
               callbacks=[tb_callback], validation_split=0.2)

%reload_ext tensorboard
%tensorboard --logdir logs